# Import packages and read CSV

In [2]:
import numpy as np
import pandas as pd
import math 
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
from scipy.optimize import minimize
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', 10)

In [15]:
data = pd.read_csv('cricket.csv')
original = data.copy()
data = data[data.Innings == 1]
data.head()

,Match,Date,Innings,Over,Runs,Total.Runs,Innings.Total.Runs,Runs.Remaining,Total.Out,Innings.Total.Out,Outs.Remaining,Wickets.in.Hand,Run.Rate,Innings.Run.Rate,Run.Rate.Required,Initial.Run.Rate.Required,Target.Score,Day-night,At.Bat,Fielding,Home.Team,Away.Team,Stadium,Country,Total.Overs,Winning.Team,Toss.Winner,at.bat.wins,at.bat.won.toss,at.bat.at.home,at.bat.bat.first,chose_bat_1st,chose_bat_2nd,forced_bat_1st,forced_bat_2nd,new.game,Error.In.Data,common.support
0,65193,14/05/1999,1,1,4,4,204,200,0,10,10,10,4.0,4.08,-1.0,-1.0,-1,0,Sri Lanka,England,England,Sri Lanka,Lord's,England,50,England,England,0,0,0,1,0,0,1,0,1,0,1
1,65193,14/05/1999,1,2,2,6,204,198,0,10,10,10,3.0,4.08,-1.0,-1.0,-1,0,Sri Lanka,England,England,Sri Lanka,Lord's,England,50,England,England,0,0,0,1,0,0,1,0,0,0,1
2,65193,14/05/1999,1,3,3,9,204,195,0,10,10,10,3.0,4.08,-1.0,-1.0,-1,0,Sri Lanka,England,England,Sri Lanka,Lord's,England,50,England,England,0,0,0,1,0,0,1,0,0,0,1
3,65193,14/05/1999,1,4,2,20,204,184,0,10,10,10,5.0,4.08,-1.0,-1.0,-1,0,Sri Lanka,England,England,Sri Lanka,Lord's,England,50,England,England,0,0,0,1,0,0,1,0,0,0,1
4,65193,14/05/1999,1,5,6,37,204,167,0,10,10,10,7.4,4.08,-1.0,-1.0,-1,0,Sri Lanka,England,England,Sri Lanka,Lord's,England,50,England,England,0,0,0,1,0,0,1,0,0,0,1


# Check if the 'Innings.Total.Runs' column is consistent

In [18]:
df = data[['Match','Innings.Total.Runs', 'Innings.Total.Runs']].copy()
df.columns = ['Match', 'TotalMax', 'TotalMin']
df = df.groupby('Match').agg({'TotalMax':'max', 'TotalMin':'min'})
df

,TotalMax,TotalMin
Match,,
64725,363,363
64726,272,272
64727,184,184
64728,206,206
64729,195,195
...,...,...
536932,418,418
536933,267,267
538069,91,91


In [19]:
df[df.TotalMax != df.TotalMin]

,TotalMax,TotalMin
Match,,


# Check if the aggregate of Runs column is equal to Innings.Total.Runs

In [64]:
df = data[['Match', 'Runs', 'Innings.Total.Runs']].copy()
df = df.groupby('Match').agg({'Runs':'sum', 'Innings.Total.Runs':'max'})
df

,Runs,Innings.Total.Runs
Match,,
64725,363,363
64726,272,272
64727,184,184
64728,205,206
64729,195,195
...,...,...
536932,418,418
536933,267,267
538069,91,91


In [65]:
# So clearly there are some inconsistencies (in 381 matches)
df['error'] = abs(df['Runs'] - df['Innings.Total.Runs'])
df[df.Runs != df['Innings.Total.Runs']]

,Runs,Innings.Total.Runs,error
Match,,,
64728,205,206,1
64731,279,280,1
64733,154,156,2
64736,280,281,1
64737,200,202,2
...,...,...,...
523482,206,208,2
530427,127,131,4
530429,256,257,1


In [66]:
errors = df.error.value_counts()
errors

0      1042
1       205
2        66
4        36
5        28
3        19
7         7
6         5
8         4
12        3
10        2
9         2
160       1
82        1
41        1
16        1
Name: error, dtype: int64

In [67]:
# Match #424849 had a error of 160 runs between aggregated column and Innings.Total.Runs column
df[df.error == 160]

,Runs,Innings.Total.Runs,error
Match,,,
424849,153,313,160


In [68]:
# Percentage of errors > 5 runs
errors_ge_5 = 0
for runs in range(6):
    errors_ge_5 += errors[runs]
errors_ge_5 = data.Match.nunique() - errors_ge_5
print('data points with error more than 5 runs: ', errors_ge_5)
print('percentage of such data points: {} %'.format(round(errors_ge_5 * 100 / data.Match.nunique(), 2)))

data points with error more than 5 runs:  27
percentage of such data points: 1.9 %


In [71]:
df[df.error > 5]

,Runs,Innings.Total.Runs,error
Match,,,
64940,310,319,9
65193,196,204,8
65203,255,261,6
65219,263,275,12
65247,249,255,6
66320,223,211,12
66325,228,221,7
66359,131,139,8
225171,295,301,6


In [72]:
# Compute list of matches with error > 5 runs
matches_error_5 = df[df.error > 5].reset_index()['Match'].unique()
matches_error_5

array([ 64940,  65193,  65203,  65219,  65247,  66320,  66325,  66359,
       225171, 236963, 249744, 249746, 259384, 267387, 349904, 360801,
       361047, 408627, 410557, 415172, 424849, 424851, 426388, 426428,
       450374, 464717, 474453])

# Analysing match 424849 (with 160 runs error)

In [61]:
df = data.copy()
df

,Match,Date,Innings,Over,Runs,Total.Runs,Innings.Total.Runs,Runs.Remaining,Total.Out,Innings.Total.Out,Outs.Remaining,Wickets.in.Hand,Run.Rate,Innings.Run.Rate,Run.Rate.Required,Initial.Run.Rate.Required,Target.Score,Day-night,At.Bat,Fielding,Home.Team,Away.Team,Stadium,Country,Total.Overs,Winning.Team,Toss.Winner,at.bat.wins,at.bat.won.toss,at.bat.at.home,at.bat.bat.first,chose_bat_1st,chose_bat_2nd,forced_bat_1st,forced_bat_2nd,new.game,Error.In.Data,common.support
0,65193,14/05/1999,1,1,4,4,204,200,0,10,10,10,4.000000,4.08,-1.0,-1.0,-1,0,Sri Lanka,England,England,Sri Lanka,Lord's,England,50,England,England,0,0,0,1,0,0,1,0,1,0,1
1,65193,14/05/1999,1,2,2,6,204,198,0,10,10,10,3.000000,4.08,-1.0,-1.0,-1,0,Sri Lanka,England,England,Sri Lanka,Lord's,England,50,England,England,0,0,0,1,0,0,1,0,0,0,1
2,65193,14/05/1999,1,3,3,9,204,195,0,10,10,10,3.000000,4.08,-1.0,-1.0,-1,0,Sri Lanka,England,England,Sri Lanka,Lord's,England,50,England,England,0,0,0,1,0,0,1,0,0,0,1
3,65193,14/05/1999,1,4,2,20,204,184,0,10,10,10,5.000000,4.08,-1.0,-1.0,-1,0,Sri Lanka,England,England,Sri Lanka,Lord's,England,50,England,England,0,0,0,1,0,0,1,0,0,0,1
4,65193,14/05/1999,1,5,6,37,204,167,0,10,10,10,7.400000,4.08,-1.0,-1.0,-1,0,Sri Lanka,England,England,Sri Lanka,Lord's,England,50,England,England,0,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126719,536933,11/12/2011,1,46,3,244,267,23,4,6,2,6,5.304348,5.34,-1.0,-1.0,-1,1,India,West Indies,India,West Indies,Chennai,India,50,India,India,1,1,1,1,1,0,0,0,0,0,1
126720,536933,11/12/2011,1,47,6,250,267,17,5,6,1,5,5.319149,5.34,-1.0,-1.0,-1,1,India,West Indies,India,West Indies,Chennai,India,50,India,India,1,1,1,1,1,0,0,0,0,0,1
126721,536933,11/12/2011,1,48,8,258,267,9,5,6,1,5,5.375000,5.34,-1.0,-1.0,-1,1,India,West Indies,India,West Indies,Chennai,India,50,India,India,1,1,1,1,1,0,0,0,0,0,1
126722,536933,11/12/2011,1,49,0,258,267,9,5,6,1,5,5.265306,5.34,-1.0,-1.0,-1,1,India,West Indies,India,West Indies,Chennai,India,50,India,India,1,1,1,1,1,0,0,0,0,0,1


In [63]:
df[df['Match'] == 424849]
# Clearly, the innings.total.runs column is wrong

,Match,Date,Innings,Over,Runs,Total.Runs,Innings.Total.Runs,Runs.Remaining,Total.Out,Innings.Total.Out,Outs.Remaining,Wickets.in.Hand,Run.Rate,Innings.Run.Rate,Run.Rate.Required,Initial.Run.Rate.Required,Target.Score,Day-night,At.Bat,Fielding,Home.Team,Away.Team,Stadium,Country,Total.Overs,Winning.Team,Toss.Winner,at.bat.wins,at.bat.won.toss,at.bat.at.home,at.bat.bat.first,chose_bat_1st,chose_bat_2nd,forced_bat_1st,forced_bat_2nd,new.game,Error.In.Data,common.support
99253,424849,12/10/2009,1,1,2,2,313,311,0,4,4,10,2.000000,6.26,-1.0,-1.0,-1,0,Zimbabwe,Kenya,Zimbabwe,Kenya,Harare,Zimbabwe,50,Zimbabwe,Zimbabwe,1,1,1,1,1,0,0,0,1,0,1
99254,424849,12/10/2009,1,2,1,3,313,310,0,4,4,10,1.500000,6.26,-1.0,-1.0,-1,0,Zimbabwe,Kenya,Zimbabwe,Kenya,Harare,Zimbabwe,50,Zimbabwe,Zimbabwe,1,1,1,1,1,0,0,0,0,0,1
99255,424849,12/10/2009,1,3,1,4,313,309,0,4,4,10,1.333333,6.26,-1.0,-1.0,-1,0,Zimbabwe,Kenya,Zimbabwe,Kenya,Harare,Zimbabwe,50,Zimbabwe,Zimbabwe,1,1,1,1,1,0,0,0,0,0,1
99256,424849,12/10/2009,1,4,13,17,313,296,0,4,4,10,4.250000,6.26,-1.0,-1.0,-1,0,Zimbabwe,Kenya,Zimbabwe,Kenya,Harare,Zimbabwe,50,Zimbabwe,Zimbabwe,1,1,1,1,1,0,0,0,0,0,1
99257,424849,12/10/2009,1,5,7,24,313,289,0,4,4,10,4.800000,6.26,-1.0,-1.0,-1,0,Zimbabwe,Kenya,Zimbabwe,Kenya,Harare,Zimbabwe,50,Zimbabwe,Zimbabwe,1,1,1,1,1,0,0,0,0,0,1
99258,424849,12/10/2009,1,6,7,31,313,282,0,4,4,10,5.166667,6.26,-1.0,-1.0,-1,0,Zimbabwe,Kenya,Zimbabwe,Kenya,Harare,Zimbabwe,50,Zimbabwe,Zimbabwe,1,1,1,1,1,0,0,0,0,0,1
99259,424849,12/10/2009,1,7,4,35,313,278,0,4,4,10,5.000000,6.26,-1.0,-1.0,-1,0,Zimbabwe,Kenya,Zimbabwe,Kenya,Harare,Zimbabwe,50,Zimbabwe,Zimbabwe,1,1,1,1,1,0,0,0,0,0,1
99260,424849,12/10/2009,1,8,8,43,313,270,0,4,4,10,5.375000,6.26,-1.0,-1.0,-1,0,Zimbabwe,Kenya,Zimbabwe,Kenya,Harare,Zimbabwe,50,Zimbabwe,Zimbabwe,1,1,1,1,1,0,0,0,0,0,1
99261,424849,12/10/2009,1,9,10,53,313,260,0,4,4,10,5.888889,6.26,-1.0,-1.0,-1,0,Zimbabwe,Kenya,Zimbabwe,Kenya,Harare,Zimbabwe,50,Zimbabwe,Zimbabwe,1,1,1,1,1,0,0,0,0,0,1
99262,424849,12/10/2009,1,10,5,58,313,255,0,4,4,10,5.800000,6.26,-1.0,-1.0,-1,0,Zimbabwe,Kenya,Zimbabwe,Kenya,Harare,Zimbabwe,50,Zimbabwe,Zimbabwe,1,1,1,1,1,0,0,0,0,0,1


# Analyse matches interrupted by rain

In [156]:
df = data.copy()
df = df.groupby('Match').agg({'Over':'max', 'Wickets.in.Hand':'min'})
df

,Over,Wickets.in.Hand
Match,,
64725,50,7
64726,50,3
64727,50,4
64728,49,1
64729,49,1
...,...,...
536932,50,5
536933,50,4
538069,31,0


In [157]:
# List of matches where first innings is possibly interrupted 
df = df[(df['Wickets.in.Hand'] > 1) & (df['Over'] < 50)]
df

,Over,Wickets.in.Hand
Match,,
64742,39,3
64751,40,4
64764,39,3
64775,45,5
64789,48,2
...,...,...
476598,42,2
483126,42,2
486522,46,2


In [158]:
df['Wickets.in.Hand'].value_counts()

2    29
3    21
4    11
5     7
6     6
7     3
9     1
8     1
Name: Wickets.in.Hand, dtype: int64

In [159]:
# We notice that there seem to be lot of matches interrupted at the 49th over
# But that's more likely the case that two wickets fell in the same over, so the wickets.in.hand went straight from 2 to zero or something
pd.set_option('display.max_rows', 10)
interruptions = df['Over'].value_counts()
print(interruptions)

49    13
48     8
47     6
32     4
45     4
      ..
27     1
44     1
23     1
21     1
20     1
Name: Over, Length: 26, dtype: int64


In [160]:
# Ignoring the matches that were interrupted after 45th over, and checking number of such interrupted matches
count_interruptions = 0
for over in range(45, 50):
    count_interruptions += interruptions[over]
count_interruptions = interruptions.sum() - count_interruptions
count_interruptions
# So there are 46 interrupted matches that need to be taken care of


46

In [161]:
interruptions.sum()

79

In [176]:
# List of matches interrupted before 45th over
# df.reset_index(inplace = True)
matches_interrupt_45 = df[df.Over < 45].reset_index()['Match'].unique()
matches_interrupt_45

array([ 64742,  64751,  64764,  64869,  64875,  64886,  64897,  64898,
        65253,  65713,  66286,  66290,  66325,  66374, 209339, 216929,
       247462, 247485, 247488, 247507, 267386, 291363, 293072, 293078,
       295786, 296917, 319135, 352669, 361046, 366623, 366710, 375460,
       377315, 406193, 424849, 426428, 433589, 446958, 450105, 455232,
       474467, 474478, 476598, 483126, 514025, 516209])

In [177]:
matches_interrupt_all = df[df.Over < 47].reset_index().Match.unique()
matches_interrupt_all

array([ 64742,  64751,  64764,  64775,  64851,  64869,  64875,  64886,
        64897,  64898,  65219,  65253,  65713,  66286,  66290,  66325,
        66374, 209339, 216929, 239915, 247462, 247485, 247488, 247507,
       267386, 291363, 293072, 293078, 295786, 296917, 319135, 352669,
       361046, 366623, 366710, 375460, 377315, 406193, 424849, 426428,
       433582, 433589, 446958, 450105, 455232, 474467, 474478, 476598,
       483126, 486522, 514025, 516209])

# Check other columns of the dataset



In [48]:
df = data.copy()
df['new.game'].value_counts()

0    66371
1     1423
Name: new.game, dtype: int64

In [ ]:
# Replace the Innings.Total.Runs

# Matches with Error.In.Data 1 

In [120]:
data['Error.In.Data'].value_counts()

0    67288
1      506
Name: Error.In.Data, dtype: int64

In [121]:
df = data.copy()
df = df.groupby('Match')['Error.In.Data'].max()
df

Match
64725     0
64726     0
64727     0
64728     0
64729     0
         ..
536932    0
536933    0
538069    0
538070    0
538071    0
Name: Error.In.Data, Length: 1423, dtype: int64

In [122]:
df.value_counts()

0    1412
1      11
Name: Error.In.Data, dtype: int64

In [143]:
matches_error_in_data = list(df[df == 1].reset_index()['Match'])

In [144]:
matches_error_in_data

[64759,
 64793,
 64858,
 64926,
 64933,
 65200,
 65618,
 66351,
 217978,
 267386,
 415172]

In [129]:
# Append match no. 410557 which had incomplete data (from 16th over)
matches_error_in_data.append(410557)
matches_error_in_data

[64759,
 64793,
 64858,
 64926,
 64933,
 65200,
 65618,
 66351,
 217978,
 267386,
 415172,
 410557]

# Prepare list of matches to be removed, and pickle it

In [169]:
print(len(matches_interrupt_all))
matches_interrupt_all

79


array([ 64742,  64751,  64764,  64775,  64789,  64851,  64869,  64875,
        64886,  64897,  64898,  64907,  64941,  65196,  65197,  65219,
        65253,  65713,  66278,  66286,  66290,  66311,  66325,  66374,
       209339, 216929, 226381, 238170, 238196, 238198, 239911, 239915,
       247462, 247477, 247485, 247488, 247507, 249748, 249756, 257770,
       267386, 267713, 282688, 282689, 291363, 293072, 293078, 295786,
       296917, 319135, 325582, 325801, 343760, 352669, 361046, 366623,
       366624, 366710, 375460, 377315, 406193, 424849, 426388, 426428,
       433582, 433589, 439147, 441783, 446958, 446963, 450105, 455232,
       474467, 474478, 476598, 483126, 486522, 514025, 516209])

In [114]:
print(len(matches_interrupt_45))
matches_interrupt_45

46


array([ 64742,  64751,  64764,  64869,  64875,  64886,  64897,  64898,
        65253,  65713,  66286,  66290,  66325,  66374, 209339, 216929,
       247462, 247485, 247488, 247507, 267386, 291363, 293072, 293078,
       295786, 296917, 319135, 352669, 361046, 366623, 366710, 375460,
       377315, 406193, 424849, 426428, 433589, 446958, 450105, 455232,
       474467, 474478, 476598, 483126, 514025, 516209])

In [115]:
print(len(matches_error_5))
matches_error_5

27


array([ 64940,  65193,  65203,  65219,  65247,  66320,  66325,  66359,
       225171, 236963, 249744, 249746, 259384, 267387, 349904, 360801,
       361047, 408627, 410557, 415172, 424849, 424851, 426388, 426428,
       450374, 464717, 474453])

In [145]:
print(len(matches_error_in_data))
matches_error_in_data

11


[64759,
 64793,
 64858,
 64926,
 64933,
 65200,
 65618,
 66351,
 217978,
 267386,
 415172]

In [178]:
matches_to_remove = set()

# for match in matches_error_5:
#     matches_to_remove.add(match)

# for match in matches_interrupt_45:
#     matches_to_remove.add(match)

for match in matches_interrupt_all:
    matches_to_remove.add(match)
    
# for match in matches_error_in_data:
#     matches_to_remove.add(match)

In [179]:
len(matches_to_remove)

52

In [182]:
# MSE 1772 and 1773 after removing 70 matches
# MSE 1759 and 1761 after removing 46 matches 
# MSE 1762 and 1763 after removing 57 matches
# MSE 1773 and 1775 after removing 79 matches
# MSE 1823 and 1825 after removing 12 matches
# MSE 1822 and 1824 after removing 11 matches
# MSE 1835 and 1837 after removing 37 matches
# MSE 1771 and 1773 after removing all interrupted matches
# MSE 1760 and 1762 after removing all interrupted matches 47

matches_to_remove = list(matches_to_remove)
print(len(matches_to_remove))
matches_to_remove

52


[64897,
 64898,
 64775,
 424849,
 66325,
 352669,
 319135,
 366623,
 291363,
 375460,
 247462,
 239915,
 433582,
 65713,
 406193,
 433589,
 476598,
 483126,
 450105,
 209339,
 426428,
 247485,
 247488,
 455232,
 65219,
 66374,
 486522,
 293072,
 64851,
 247507,
 296917,
 293078,
 361046,
 216929,
 377315,
 474467,
 64869,
 64742,
 65253,
 514025,
 295786,
 64875,
 66286,
 64751,
 446958,
 474478,
 66290,
 516209,
 64886,
 366710,
 267386,
 64764]

In [183]:
matches_to_remove

[64897,
 64898,
 64775,
 424849,
 66325,
 352669,
 319135,
 366623,
 291363,
 375460,
 247462,
 239915,
 433582,
 65713,
 406193,
 433589,
 476598,
 483126,
 450105,
 209339,
 426428,
 247485,
 247488,
 455232,
 65219,
 66374,
 486522,
 293072,
 64851,
 247507,
 296917,
 293078,
 361046,
 216929,
 377315,
 474467,
 64869,
 64742,
 65253,
 514025,
 295786,
 64875,
 66286,
 64751,
 446958,
 474478,
 66290,
 516209,
 64886,
 366710,
 267386,
 64764]

In [180]:
import pickle

with open('matches_to_remove.pkl', 'wb') as f:
    pickle.dump(matches_to_remove, f)

In [110]:
with open('matches_to_remove.pkl', 'rb') as f:
    loaded = pickle.load(f)

In [111]:
loaded

[64897,
 64898,
 66320,
 249744,
 249746,
 225171,
 424849,
 66325,
 424851,
 426388,
 352669,
 319135,
 366623,
 236963,
 291363,
 375460,
 247462,
 65193,
 64940,
 65713,
 406193,
 65203,
 408627,
 433589,
 476598,
 66359,
 259384,
 450105,
 483126,
 209339,
 426428,
 410557,
 247485,
 247488,
 455232,
 65219,
 415172,
 450374,
 66374,
 464717,
 349904,
 293072,
 247507,
 474453,
 293078,
 361047,
 296917,
 361046,
 65247,
 360801,
 216929,
 377315,
 474467,
 64869,
 64742,
 65253,
 514025,
 295786,
 64875,
 66286,
 64751,
 446958,
 474478,
 66290,
 516209,
 64886,
 366710,
 267386,
 267387,
 64764]